In [1]:
import json
import os
from typing import Dict, List, Tuple, Union

import mlx.optimizers as optim
from mlx.utils import tree_flatten
from mlx_lm import load
from mlx_lm.tuner import TrainingArgs, linear_to_lora_layers, train

In [2]:
model_path = "mlx-community/Phi-3.5-mini-instruct-4bit"
model, tokenizer = load(model_path)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

In [3]:
adapter_path = "adapters"
os.makedirs(adapter_path, exist_ok=True)
adapter_config_path = os.path.join(adapter_path, "adapter_config.json")
adapter_file_path = os.path.join(adapter_path, "adapters.safetensors")

# Lora config

In [4]:
lora_config = {
    "num_layers": 8,
    "lora_parameters": {
        "rank": 4,
        "scale": 20.0,
        "dropout": 0.0,
    },
}

In [5]:
with open(adapter_config_path, "w") as f:
    json.dump(lora_config, f, indent=4)

In [6]:
training_args = TrainingArgs(
    adapter_file=adapter_file_path,
    iters=100,
    steps_per_eval=50,
    grad_checkpoint=True,
)

In [7]:
model.freeze()
linear_to_lora_layers(model, lora_config["num_layers"], lora_config["lora_parameters"])
num_train_params = sum(v.size for _, v in tree_flatten(model.trainable_parameters()))
print(f"Number of trainable parameters: {num_train_params}")
model.train()

Number of trainable parameters: 393216


Model(
  (model): Phi3Model(
    (embed_tokens): QuantizedEmbedding(32064, 3072, group_size=64, bits=4)
    (layers.0): TransformerBlock(
      (self_attn): Attention(
        (qkv_proj): QuantizedLinear(input_dims=3072, output_dims=9216, bias=False, group_size=64, bits=4)
        (o_proj): QuantizedLinear(input_dims=3072, output_dims=3072, bias=False, group_size=64, bits=4)
        (rope): SuScaledRoPE()
      )
      (mlp): MLP(
        (gate_up_proj): QuantizedLinear(input_dims=3072, output_dims=16384, bias=False, group_size=64, bits=4)
        (down_proj): QuantizedLinear(input_dims=8192, output_dims=3072, bias=False, group_size=64, bits=4)
      )
      (input_layernorm): RMSNorm(3072, eps=1e-05)
      (post_attention_layernorm): RMSNorm(3072, eps=1e-05)
    )
    (layers.1): TransformerBlock(
      (self_attn): Attention(
        (qkv_proj): QuantizedLinear(input_dims=3072, output_dims=9216, bias=False, group_size=64, bits=4)
        (o_proj): QuantizedLinear(input_dims=3072, out

In [8]:
class Metrics:
    def __init__(self) -> None:
        self.train_losses: List[Tuple[int, float]] = []
        self.val_losses: List[Tuple[int, float]] = []

    def on_train_loss_report(self, info: Dict[str, Union[float, int]]) -> None:
        self.train_losses.append((info["iteration"], info["train_loss"]))

    def on_val_loss_report(self, info: Dict[str, Union[float, int]]) -> None:
        self.val_losses.append((info["iteration"], info["val_loss"]))

In [12]:
metrics = Metrics()

# load data

In [9]:
from mlx_lm.tuner.datasets import load_hf_dataset
config = { }
train_set, val_set, test_set = load_hf_dataset(
    data_id="mlx-community/wikisql",
    tokenizer=tokenizer,
    config=config,
)

In [10]:
print(f"Test set size: {len(test_set)}")
print(f"Validation set size: {len(val_set)}")
print(f"Training set size: {len(train_set)}")
print(f"test set: {test_set[:2]}")

Test set size: 100
Validation set size: 100
Training set size: 1000
test set: {'text': ["table: 1-10015132-16\ncolumns: Player, No., Nationality, Position, Years in Toronto, School/Club Team\nQ: What is terrence ross' nationality\nA: SELECT Nationality FROM 1-10015132-16 WHERE Player = 'Terrence Ross'", "table: 1-10015132-16\ncolumns: Player, No., Nationality, Position, Years in Toronto, School/Club Team\nQ: What clu was in toronto 1995-96\nA: SELECT School/Club Team FROM 1-10015132-16 WHERE Years in Toronto = '1995-96'"]}


In [13]:
from mlx_lm.tuner.datasets import CacheDataset

train_dataset = CacheDataset(train_set)
val_dataset = CacheDataset(val_set)

train(
    model,
    optim.Adam(learning_rate=1e-5),
    train_dataset,
    val_dataset,
    args=training_args,
    training_callback=metrics
)


Starting training..., iters: 100


Calculating loss...: 100%|██████████| 25/25 [00:40<00:00,  1.60s/it]

Iter 1: Val loss 3.253, Val took 40.017s


Iter 10: Train loss 3.307, Learning Rate 1.000e-05, It/sec 0.459, Tokens/sec 179.449, Trained Tokens 3909, Peak mem 3.126 GB
Iter 20: Train loss 2.921, Learning Rate 1.000e-05, It/sec 0.437, Tokens/sec 178.133, Trained Tokens 7988, Peak mem 3.126 GB
Iter 30: Train loss 2.336, Learning Rate 1.000e-05, It/sec 0.423, Tokens/sec 172.896, Trained Tokens 12071, Peak mem 3.126 GB
Iter 40: Train loss 2.251, Learning Rate 1.000e-05, It/sec 0.449, Tokens/sec 170.352, Trained Tokens 15864, Peak mem 3.126 GB


Calculating loss...: 100%|██████████| 25/25 [00:43<00:00,  1.74s/it]

Iter 50: Val loss 1.907, Val took 43.516s


Iter 50: Train loss 2.027, Learning Rate 1.000e-05, It/sec 0.451, Tokens/sec 171.264, Trained Tokens 19664, Peak mem 3.126 GB
Iter 60: Train loss 1.967, Learning Rate 1.000e-05, It/sec 0.369, Tokens/sec 161.988, Trained Tokens 24058, Peak mem 3.291 GB
Iter 70: Train loss 1.639, Learning Rate 1.000e-05, It/sec 0.380, Tokens/sec 150.717, Trained Tokens 28026, Peak mem 3.291 GB
Iter 80: Train loss 1.563, Learning Rate 1.000e-05, It/sec 0.406, Tokens/sec 156.848, Trained Tokens 31889, Peak mem 3.291 GB
Iter 90: Train loss 1.549, Learning Rate 1.000e-05, It/sec 0.368, Tokens/sec 143.172, Trained Tokens 35775, Peak mem 3.291 GB


Calculating loss...: 100%|██████████| 25/25 [00:46<00:00,  1.88s/it]

Iter 100: Val loss 1.540, Val took 46.953s


Iter 100: Train loss 1.418, Learning Rate 1.000e-05, It/sec 0.351, Tokens/sec 143.131, Trained Tokens 39849, Peak mem 3.291 GB
Iter 100: Saved adapter weights to adapters/adapters.safetensors and adapters/0000100_adapters.safetensors.
Saved final weights to adapters/adapters.safetensors.


## Fusionar modelo base con adaptador

In [1]:
! python -m mlx_lm fuse  --model ./phi3-mlx --adapter-path ./adapters --save-path ./new_phi3-mlx

Loading pretrained model


In [ ]:
! pip install huggingface_hub
! pip install ipywidgets

# Subir modelo a HF
Utilizando el API de HF se sube el modelo a deimagjas/Phi-3.5-mini-instruct-4bit-sft

In [ ]:
from huggingface_hub import login

login()

In [ ]:
from huggingface_hub import create_repo, upload_folder

repo_id = "deimagjas/Phi-3.5-mini-instruct-4bit-sft"

upload_folder(
    folder_path="./new_phi3-mlx",
    repo_id=repo_id
)
